In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf
import spacy

2023-06-20 23:16:28.621659: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-20 23:16:28.623286: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-20 23:16:28.657093: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-20 23:16:28.657688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 23:16:29.356877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Load test data

In [2]:
test_df = pd.read_csv("/home/jovyan/work/data/health/lstm/test.csv")

In [43]:
test_df

,sentence_idx,word,tag
0,1,BP,O
1,1,Record,O
2,1,.,O
3,1,presents,O
4,1,u,O
...,...,...,...
126151,61231,-,I-PER
126152,61231,Ora,B-PER
126153,61231,-,I-PER
126154,61231,MD,O


### Load lstm Model

In [3]:
lstm_dir = "/home/jovyan/work/data/output/LSTM_health_b10_s4_b32"
lstm_model = tf.keras.models.load_model(lstm_dir)

In [40]:
y_pred = lstm_model.predict(test_df["word"])
y_pred

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

### Load spaCy Model

In [32]:
spacy_dir =  "/home/jovyan/work/data/output/spacyModel-health-epoch30-GDPR"
nlp_spacy = spacy.load(spacy_dir)

In [34]:
# Empty lists to store predictions and true tags
y_pred_spacy = []
y_true = test_df['tag'].tolist()

In [38]:
import pandas as pd

# Create empty lists to store predictions and true tags
y_pred_spacy = []
y_true = []

# Prepare the texts for batch processing
texts = [word for word in test_df['word'] if not pd.isnull(word) and word != '']

# Process the texts using the loaded spaCy model
docs = nlp_spacy.pipe(texts)

# Iterate over each processed document
for doc in docs:
    # Get the predicted tag or assign a placeholder value
    predicted_tag = doc[0].tag_ if doc else 'UNKNOWN'

    # Add the predicted tag to the y_pred_spacy list
    y_pred_spacy.append(predicted_tag)

# Create the results dataframe
results = pd.DataFrame({'word': texts, 'y_pred_spacy': y_pred_spacy, 'y_true': y_true})

# Print the results
print(results)


ValueError: All arrays must be of the same length

In [ ]:
results_df = pd.DataFrame({'word': test_df['word'], 'y_pred': y_pred_spacy, 'y_true': y_true})